# For Developer Only!

In [2]:
# Run Once
!git clone https://github.com/raixxx-thesis-git/outer_realm --branch dev

Cloning into 'outer_realm'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 177 (delta 105), reused 124 (delta 52), pack-reused 0
Receiving objects: 100% (177/177), 31.05 KiB | 1.63 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [3]:
import outer_realm as otr
import shutil
import sys

In [6]:
def otr_update(otr):
  pending_for_deletion = []
  for i in sys.modules:
    if i.split('.')[0] == 'outer_realm':
      pending_for_deletion.append(i)

  for i in pending_for_deletion:
    del sys.modules[i]

  import outer_realm as otr
  return otr

In [5]:
# Run to update
%cd outer_realm
!git stash
!git fetch --all
!git pull
!git checkout dev
%cd ..

otr = otr_update(otr)

/content/outer_realm
Saved working directory and index state WIP on dev: 6ff38ef debug
Fetching origin
Already up to date.
Already on 'dev'
Your branch is up to date with 'origin/dev'.
/content


In [6]:
!rm -rf train_dataset
!rm -rf val_dataset

In [7]:
location_template = 'drive/MyDrive/thesis/tfrecords_stalta_threadX(coreY)'

def get_copy_paths(copy_codes):
  copy_paths = []
  for copy_code in copy_codes:
    target_location = location_template.replace('X', copy_code[0]).replace('Y', copy_code[1])
    copy_paths.append(target_location)
  return copy_paths

copy_codes = ['00', '01', '02']
train_dataset_to_copy_paths = get_copy_paths(copy_codes)

copy_codes = ['10', '11', '12']
val_dataset_to_copy_paths = get_copy_paths(copy_codes)


In [8]:
shutil.copytree(train_dataset_to_copy_paths[0], 'train_dataset')
shutil.copytree(val_dataset_to_copy_paths[0], 'val_dataset')

'val_dataset'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Start Here

In [1]:
import os
import tensorflow as tf
import outer_realm as otr

from tensorflow.keras.layers import Input, Dense, Conv1D, LSTM, \
MaxPooling1D, Flatten

# Load Dataset

Recommended: Put all of your training tfrecords in ```train_dataset``` folder and all of your validation tfrecords in ```val_dataset``` folder. Warning: Your dataset must be in a flat hierarchy (no folder inside the dataset folder).

In [2]:
train_dataset_lists = [f'train_dataset/{i}' for i in os.listdir('train_dataset')]
train_dataset = otr.initium.reader_get_data_and_epicenter(train_dataset_lists, True)

val_dataset_lists = [f'val_dataset/{i}' for i in os.listdir('val_dataset')]
val_dataset = otr.initium.reader_get_data_and_epicenter(val_dataset_lists, True)

# Define Your Model

Define your model here.

In [3]:
inputs = tf.keras.layers.Input(shape=(6400, 3))
x = Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)

for _ in range(2):
  x = Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu')(x)
  x = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2)(x)

x = tf.keras.layers.Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPooling1D(pool_size=3, strides=2)(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(1)(x)

proposed_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Call Apex Module

In [4]:
import outer_realm as otr
import sys
import os

Configure Apex Module. I think the following arguments are straightforward enough to understand. No further explanation is required. Info: I have created an auto-checker for model compatibility and dataset compatibility. An error will be returned if any incompatibilities occur.

In [31]:
apex = otr.Apex(training_dataset=train_dataset,
                validation_dataset=val_dataset,
                window_length=6400,
                channel_size=3,
                batch_size=365,
                model=proposed_model,
                epoch=3)

Please wait. Checking model compability.
Done! Your model is compatible.
Please wait. Checking dataset compability.
Done! Your dataset is compatible.


We assign the optimizers.

In [32]:
apex.update_optimizer(tf.keras.optimizers.Adam(learning_rate=1e-4))

Run the following code to train your model. Each time you train, an ```ApexTrainer``` object is created and a new training session ID is generated. You can see your models and its metadata (epoch logs) in the folder that is named based on your training session id.

In [34]:
'''
if calculate_r2_per_epoch is false, the validation system won't calcualte R2 score
if save_model_per_epoch is false, the model won't be saved per epoch
(only be saved at the end of the training)
'''
apex.train(save_model_per_epoch=True, calculate_r2_per_epoch=True)

Your training session ID: 91f9c40c-6b54-4e1a-a1ed-e9a4a6f0ec10.
All training logs and model will automatically be saved in 91f9c40c-6b54-4e1a-a1ed-e9a4a6f0ec10 folder.

Entering training stage now.
Note: Proper progress bar appears at the second epoch.

Epoch 1/3


|                              | [02:11<00:00] Batch 211/211 | Training Loss: 4014.9402


Validation Loss: 16882.6504
Validation R2: 0.4521%

Epoch 2/3


|██████████████████████████████| [02:03<00:00] Batch 211/212 | Training Loss: 4151.3291


Validation Loss: 16540.0430
Validation R2: 0.4662%

Epoch 3/3


|██████████████████████████████| [02:03<00:00] Batch 211/212 | Training Loss: 4130.7891


Validation Loss: 16098.1660
Validation R2: 0.4749%

Closed training session, Apex Trainer is freed.


You can also update your previously assigned configuration.

In [35]:
apex.update_epoch(5)

Now let's retrai again. This time we are training with 5 epochs.

In [36]:
apex.train(save_model_per_epoch=False)

Your training session ID: 5dc539c3-573f-44c0-89c2-943c0cb5c04a.
All training logs and model will automatically be saved in 5dc539c3-573f-44c0-89c2-943c0cb5c04a folder.

Entering training stage now.
Note: Proper progress bar appears at the second epoch.

Epoch 1/5


|                              | [02:09<00:00] Batch 211/211 | Training Loss: 3953.6675


Validation Loss: 15676.1729
Validation R2: 0.4990%

Epoch 2/5


|██████████████████████████████| [02:03<00:00] Batch 211/212 | Training Loss: 3878.1223


Validation Loss: 15300.2305
Validation R2: 0.5219%

Epoch 3/5


...

Validation Loss: 14877.4561
Validation R2: 0.5483%

Epoch 4/5


|██████████████████████████████| [02:03<00:00] Batch 211/212 | Training Loss: 2639.1931


Validation Loss: 14541.0850
Validation R2: 0.5801%

Epoch 5/5


|██████████████████████████████| [02:03<00:00] Batch 211/212 | Training Loss: 2295.9592


Validation Loss: 14209.8252
Validation R2: 0.5970%

Closed training session, Apex Trainer is freed.
